In [1]:
import xarray as xr
import numpy as np
import datetime

# Unification of MODIS chlorophyll files
# (do not run!)

In [4]:
temp = xr.open_dataset("../data/CAL/CMEMS_temp_extended.nc").analysed_sst

lons, lats = temp.lon, temp.lat

In [5]:
chl = xr.open_mfdataset("../data/CAL/MODIS_DATA_DAILY/AQUA_MODIS.*.L3m.DAY.CHL.x_chlor_a.nc", combine = "nested", concat_dim="time")

In [11]:
# BASH COMMAND TO USE FOR dates.txt CREATION
# printf '%s\n' * | cut -d . -f2 >> dates.txt

with open('../data/CAL/MODIS_DATA_DAILY/dates.txt', 'rb') as file:
     data = file.read()


dates_list = data.decode("utf-16").split("\r\n")

In [12]:
dates_list[-1]

'20211231'

In [13]:
chl["time"] = [datetime.datetime.strptime(i, "%Y%m%d") for i in dates_list]
chl = chl.interp(lon = lons, lat = lats, method = "nearest")

In [14]:
chl.load().to_netcdf("../data/CAL/MODIS_chl_extended.nc")

In [28]:
chl = xr.open_dataset("../data/CAL/MODIS_chl_extended.nc").convert_calendar("noleap")

datetimeindex = chl.indexes['time'].to_datetimeindex()

chl["time"] = datetimeindex

c:\Users\feder\miniconda3\lib\site-packages\xarray\coding\times.py:351: FutureWarning: Index.ravel returning ndarray is deprecated; in a future version this will return a view on self.
  sample = dates.ravel()[0]
C:\Users\feder\AppData\Local\Temp\ipykernel_23432\40185623.py:3: RuntimeWarning: Converting a CFTimeIndex with dates from a non-standard calendar, 'noleap', to a pandas.DatetimeIndex, which uses dates from the standard calendar.  This may lead to subtle errors in operations that depend on the length of time between dates.
  datetimeindex = chl.indexes['time'].to_datetimeindex()


In [29]:
chl

<xarray.Dataset>
Dimensions:  (time: 6920, lat: 301, lon: 240, rgb: 3, eightbitcolor: 256)
Coordinates:
  * time     (time) datetime64[ns] 2003-01-01 2003-01-02 ... 2021-12-31
  * lon      (lon) float32 -130.0 -129.9 -129.9 -129.8 ... -118.1 -118.1 -118.0
  * lat      (lat) float32 30.02 30.08 30.12 30.17 ... 44.88 44.92 44.97 45.03
Dimensions without coordinates: rgb, eightbitcolor
Data variables:
    chlor_a  (time, lat, lon) float32 nan nan nan nan nan ... nan nan nan nan
    palette  (time, rgb, eightbitcolor) float64 147.0 0.0 108.0 ... 0.0 0.0 0.0
Attributes: (12/61)
    product_name:                     AQUA_MODIS.20030101.L3m.DAY.CHL.x_chlor...
    instrument:                       MODIS
    title:                            MODISA Level-3 Equidistant Cylindrical ...
    project:                          Ocean Biology Processing Group (NASA/GS...
    platform:                         Aqua
    source:                           satellite observations from MODIS-Aqua
    ...                               ...
    processing_level:                 L3 Mapped
    cdm_data_type:                    grid
    proj4_string:                     +proj=eqc +lat_ts=0 +lat_0=0 +x_0=0 +y_...
    data_bins:                        31497
    data_minimum:                     0.051318873
    data_maximum:                     14.255431

In [55]:
# chl.load().to_netcdf("../data/CAL/MODIS_chl_extended_timeprocessed.nc")

# Computation of chlorophyll and temperature climatologies

In [18]:
chl = xr.open_dataset("../data/CAL/MODIS_chl_extended.nc").chlor_a
temp = xr.open_dataset("../data/CAL/CMEMS_temp_extended.nc").analysed_sst

lons, lats = temp.lon, temp.lat

In [19]:
chl_clima = chl.groupby("time.dayofyear").mean()

temp_clima = temp.groupby("time.dayofyear").mean()

In [20]:
chl_std = chl.groupby("time.dayofyear").std(skipna = True)

temp_std = temp.groupby("time.dayofyear").std(skipna = True)

c:\Users\feder\miniconda3\lib\site-packages\numpy\lib\nanfunctions.py:1879: RuntimeWarning: Degrees of freedom <= 0 for slice.
  var = nanvar(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
c:\Users\feder\miniconda3\lib\site-packages\numpy\lib\nanfunctions.py:1879: RuntimeWarning: Degrees of freedom <= 0 for slice.
  var = nanvar(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
c:\Users\feder\miniconda3\lib\site-packages\numpy\lib\nanfunctions.py:1879: RuntimeWarning: Degrees of freedom <= 0 for slice.
  var = nanvar(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
c:\Users\feder\miniconda3\lib\site-packages\numpy\lib\nanfunctions.py:1879: RuntimeWarning: Degrees of freedom <= 0 for slice.
  var = nanvar(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
c:\Users\feder\miniconda3\lib\site-packages\numpy\lib\nanfunctions.py:1879: RuntimeWarning: Degrees of freedom <= 0 for slice.
  var = nanvar(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
c:\Users\feder\miniconda3\lib\site-packages\numpy\lib\n

In [21]:
days_list = [datetime.datetime.strptime(str(2016) + "-" + str(chl_clima.dayofyear.to_numpy()[i]), "%Y-%j").strftime("%d-%m") for i in range(0, 366)]

chl_clima["dayofyear"] = days_list
temp_clima["dayofyear"] = days_list

chl_std["dayofyear"] = days_list
temp_std["dayofyear"] = days_list

In [30]:
chl_clima.to_netcdf("../data/CAL/chl_climatology.nc")
chl_std.to_netcdf("../data/CAL/chl_std.nc")

In [ ]:
temp_clima.to_netcdf("../data/CAL/temp_climatology.nc")
temp_std.to_netcdf("../data/CAL/temp_std.nc")

# Bathymetry interpolation

In [6]:
bathy = xr.open_dataset("../data/CAL/bathymetry.nc")

bathy = bathy.interp(lon = lons, lat = lats, method = "nearest")

bathy.to_netcdf("../data/CAL/bathymetry_interpolated.nc")

# Time uniformation for temperature

In [15]:
temp = xr.open_dataset("../data/CAL/CMEMS_temp_extended.nc")#.analysed_sst
temp["time"] = temp["time"].dt.strftime("%Y-%m-%d").astype(np.datetime64)

temp.analysed_sst.load().to_netcdf("../data/CAL/CMEMS_temp_extended.nc")

temp

In [ ]:
xr.open_dataset("../data/CAL/CMEMS_temp_extended.nc")#.analysed_sst

# 8-day averaged time series

In [21]:
chlorophyll = xr.open_dataset("../data/CAL/MODIS_chl_extended.nc").convert_calendar("noleap")

chlorophyll = chlorophyll.sel(time=~((chlorophyll.time.dt.month == 12) & (chlorophyll.time.dt.day >= 27))).resample(time = "1D").asfreq()

c = chlorophyll.where(chlorophyll["time.year"] == 2003, drop = True).resample(time = "8D").mean()
for y in range(2004,2022):
    c = xr.merge([c, chlorophyll.where(chlorophyll["time.year"] == y, drop = True).resample(time = "8D").mean()])

c = c.chlor_a

datetimeindex = c.indexes['time'].to_datetimeindex()

c["time"] = datetimeindex

c:\Users\feder\miniconda3\lib\site-packages\xarray\coding\times.py:351: FutureWarning: Index.ravel returning ndarray is deprecated; in a future version this will return a view on self.
  sample = dates.ravel()[0]
C:\Users\feder\AppData\Local\Temp\ipykernel_23432\2385432182.py:11: RuntimeWarning: Converting a CFTimeIndex with dates from a non-standard calendar, 'noleap', to a pandas.DatetimeIndex, which uses dates from the standard calendar.  This may lead to subtle errors in operations that depend on the length of time between dates.
  datetimeindex = c.indexes['time'].to_datetimeindex()


In [22]:
c.load().to_netcdf("../data/CAL/MODIS_chl_extended_8D.nc")

In [10]:
temperature = xr.open_dataset("../data/CAL/CMEMS_temp_extended.nc").convert_calendar("noleap")

temperature = temperature.sel(time=~((temperature.time.dt.month == 12) & (temperature.time.dt.day >= 27))).resample(time = "1D").asfreq()

t = temperature.where(temperature["time.year"] == 2003, drop = True).resample(time = "8D").mean()
for y in range(2004,2022):
    t = xr.merge([t, temperature.where(temperature["time.year"] == y, drop = True).resample(time = "8D").mean()])

t = t.analysed_sst

In [11]:
datetimeindex = t.indexes['time'].to_datetimeindex()

t["time"] = datetimeindex

c:\Users\feder\miniconda3\lib\site-packages\xarray\coding\times.py:351: FutureWarning: Index.ravel returning ndarray is deprecated; in a future version this will return a view on self.
  sample = dates.ravel()[0]
C:\Users\feder\AppData\Local\Temp\ipykernel_20924\2651879082.py:1: RuntimeWarning: Converting a CFTimeIndex with dates from a non-standard calendar, 'noleap', to a pandas.DatetimeIndex, which uses dates from the standard calendar.  This may lead to subtle errors in operations that depend on the length of time between dates.
  datetimeindex = t.indexes['time'].to_datetimeindex()


In [13]:
t.load().to_netcdf("../data/CAL/CMEMS_temp_extended_8D.nc")

In [4]:
# chl = xr.open_mfdataset("../data/SCS/MODIS_DATA_8D/*.nc", combine = "nested", concat_dim="time")

In [13]:
# BASH COMMAND TO USE FOR dates.txt CREATION
# printf '%s\n' * | cut -d . -f2 >> dates.txt

# with open('../data/SCS/MODIS_DATA_8D/dates.txt', 'rb') as file:
#      data = file.read()


# dates_list = data.decode("utf-16").split("\r\n")

In [16]:
# chl["time"] = [datetime.datetime.strptime(i, "%Y%m%d") for i in dates_list]
# chl = chl.interp(lon = lons, lat = lats, method = "nearest")

# chl.chlor_a.load().to_netcdf("../data/SCS/MODIS_chl_extended_8D.nc")

# Creation of anomalies files

In [11]:
#chlorophyll = xr.open_dataset("../data/SCS/MODIS_chl_extended_8D.nc").chlor_a
temperature = xr.open_dataset("../data/SCS/CMEMS_temp_extended_8D.nc").analysed_sst

#chlorophyll = chlorophyll.assign_coords(day_of_year = chlorophyll.time.dt.strftime("%d-%m"))
#chl_anomalies = ((chlorophyll.groupby("day_of_year") - chlorophyll.groupby("day_of_year").mean("time")).groupby("day_of_year")) / chlorophyll.groupby("day_of_year").std("time")

temperature = temperature.assign_coords(day_of_year = temperature.time.dt.strftime("%d-%m"))
temp_anomalies = ((temperature.groupby("day_of_year") - temperature.groupby("day_of_year").mean("time")).groupby("day_of_year")) / temperature.groupby("day_of_year").std("time")

c:\Users\feder\miniconda3\lib\site-packages\numpy\lib\nanfunctions.py:1879: RuntimeWarning: Degrees of freedom <= 0 for slice.
  var = nanvar(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
c:\Users\feder\miniconda3\lib\site-packages\numpy\lib\nanfunctions.py:1879: RuntimeWarning: Degrees of freedom <= 0 for slice.
  var = nanvar(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
c:\Users\feder\miniconda3\lib\site-packages\numpy\lib\nanfunctions.py:1879: RuntimeWarning: Degrees of freedom <= 0 for slice.
  var = nanvar(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
c:\Users\feder\miniconda3\lib\site-packages\numpy\lib\nanfunctions.py:1879: RuntimeWarning: Degrees of freedom <= 0 for slice.
  var = nanvar(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
c:\Users\feder\miniconda3\lib\site-packages\numpy\lib\nanfunctions.py:1879: RuntimeWarning: Degrees of freedom <= 0 for slice.
  var = nanvar(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
c:\Users\feder\miniconda3\lib\site-packages\numpy\lib\n

In [8]:
temperature = xr.open_dataset("../data/SCS/MODIS_temp_extended_8D.nc").sst

temperature = temperature.assign_coords(day_of_year = temperature.time.dt.strftime("%d-%m"))
temp_anomalies = ((temperature.groupby("day_of_year") - temperature.groupby("day_of_year").mean("time")).groupby("day_of_year")) / temperature.groupby("day_of_year").std("time")

c:\Users\feder\miniconda3\lib\site-packages\numpy\lib\nanfunctions.py:1879: RuntimeWarning: Degrees of freedom <= 0 for slice.
  var = nanvar(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
c:\Users\feder\miniconda3\lib\site-packages\numpy\lib\nanfunctions.py:1879: RuntimeWarning: Degrees of freedom <= 0 for slice.
  var = nanvar(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
c:\Users\feder\miniconda3\lib\site-packages\numpy\lib\nanfunctions.py:1879: RuntimeWarning: Degrees of freedom <= 0 for slice.
  var = nanvar(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
c:\Users\feder\miniconda3\lib\site-packages\numpy\lib\nanfunctions.py:1879: RuntimeWarning: Degrees of freedom <= 0 for slice.
  var = nanvar(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
c:\Users\feder\miniconda3\lib\site-packages\numpy\lib\nanfunctions.py:1879: RuntimeWarning: Degrees of freedom <= 0 for slice.
  var = nanvar(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
c:\Users\feder\miniconda3\lib\site-packages\numpy\lib\n

In [10]:
#chl_anomalies.load().to_netcdf("../data/SCS/MODIS_chl_anomalies_8D.nc")
temp_anomalies.load().to_netcdf("../data/SCS/CMEMS_temp_anomalies_8D.nc")

In [4]:
chlorophyll = xr.open_dataset("../data/CAL/MODIS_chl_extended_8D.nc").chlor_a
temperature = xr.open_dataset("../data/CAL/CMEMS_temp_extended_8D.nc").analysed_sst

chlorophyll = chlorophyll.assign_coords(day_of_year = chlorophyll.time.dt.strftime("%d-%m"))
chl_anomalies = ((chlorophyll.groupby("day_of_year") - chlorophyll.groupby("day_of_year").mean("time")).groupby("day_of_year")) / chlorophyll.groupby("day_of_year").std("time")

temperature = temperature.assign_coords(day_of_year = temperature.time.dt.strftime("%d-%m"))
temp_anomalies = ((temperature.groupby("day_of_year") - temperature.groupby("day_of_year").mean("time")).groupby("day_of_year")) / temperature.groupby("day_of_year").std("time")

c:\Users\feder\miniconda3\lib\site-packages\numpy\lib\nanfunctions.py:1879: RuntimeWarning: Degrees of freedom <= 0 for slice.
  var = nanvar(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
c:\Users\feder\miniconda3\lib\site-packages\numpy\lib\nanfunctions.py:1879: RuntimeWarning: Degrees of freedom <= 0 for slice.
  var = nanvar(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
c:\Users\feder\miniconda3\lib\site-packages\numpy\lib\nanfunctions.py:1879: RuntimeWarning: Degrees of freedom <= 0 for slice.
  var = nanvar(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
c:\Users\feder\miniconda3\lib\site-packages\numpy\lib\nanfunctions.py:1879: RuntimeWarning: Degrees of freedom <= 0 for slice.
  var = nanvar(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
c:\Users\feder\miniconda3\lib\site-packages\numpy\lib\nanfunctions.py:1879: RuntimeWarning: Degrees of freedom <= 0 for slice.
  var = nanvar(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
c:\Users\feder\miniconda3\lib\site-packages\numpy\lib\n

In [5]:
chl_anomalies.load().to_netcdf("../data/CAL/MODIS_chl_anomalies_8D.nc")
temp_anomalies.load().to_netcdf("../data/CAL/CMEMS_temp_anomalies_8D.nc")

# MODIS SST

In [2]:
path = "../data/SCS/"

In [8]:
temp = xr.open_mfdataset(path + "MODIS_TEMP_DAILY/AQUA_MODIS.*.L3m.DAY.SST.x_sst.nc", combine ="nested", concat_dim="time")

In [9]:
# BASH COMMAND TO USE FOR dates.txt CREATION
# printf '%s\n' * | cut -d . -f2 >> dates.txt

with open(path + "MODIS_TEMP_DAILY/dates.txt", 'rb') as file:
     data = file.read()


dates_list = data.decode("utf-16").split("\r\n")

In [12]:
temp

<xarray.Dataset>
Dimensions:   (time: 6826, lat: 240, lon: 144, rgb: 3, eightbitcolor: 256)
Coordinates:
  * lat       (lat) float32 18.98 18.94 18.9 18.85 ... 9.146 9.104 9.062 9.021
  * lon       (lon) float32 108.0 108.1 108.1 108.1 ... 113.9 113.9 113.9 114.0
Dimensions without coordinates: time, rgb, eightbitcolor
Data variables:
    sst       (time, lat, lon) float32 dask.array<chunksize=(1, 240, 144), meta=np.ndarray>
    qual_sst  (time, lat, lon) float32 dask.array<chunksize=(1, 240, 144), meta=np.ndarray>
    palette   (time, rgb, eightbitcolor) uint8 dask.array<chunksize=(1, 3, 256), meta=np.ndarray>
Attributes: (12/61)
    product_name:                     AQUA_MODIS.20030101.L3m.DAY.SST.x_sst.nc
    instrument:                       MODIS
    title:                            MODISA Level-3 Equidistant Cylindrical ...
    project:                          Ocean Biology Processing Group (NASA/GS...
    platform:                         Aqua
    source:                           satellite observations from MODIS-Aqua
    ...                               ...
    processing_level:                 L3 Mapped
    cdm_data_type:                    grid
    proj4_string:                     +proj=eqc +lat_ts=0 +lat_0=0 +x_0=0 +y_...
    data_bins:                        11567
    data_minimum:                     21.74
    data_maximum:                     28.5775

In [14]:
temp["time"] = [datetime.datetime.strptime(i, "%Y%m%d") for i in dates_list]
#temp = temp.interp(lon = lons, lat = lats, method = "nearest")

In [15]:
temp.load().to_netcdf(path + "MODIS_temp_extended.nc")

In [4]:
temp = xr.open_dataset(path + "MODIS_temp_extended.nc").load().convert_calendar("noleap", dim = "time")

temp

<xarray.Dataset>
Dimensions:   (time: 6821, lat: 240, lon: 144, rgb: 3, eightbitcolor: 256)
Coordinates:
  * lat       (lat) float32 18.98 18.94 18.9 18.85 ... 9.146 9.104 9.062 9.021
  * lon       (lon) float32 108.0 108.1 108.1 108.1 ... 113.9 113.9 113.9 114.0
  * time      (time) object 2003-01-01 00:00:00 ... 2021-12-31 00:00:00
Dimensions without coordinates: rgb, eightbitcolor
Data variables:
    sst       (time, lat, lon) float32 23.79 23.7 23.6 23.57 ... nan nan nan nan
    qual_sst  (time, lat, lon) float32 0.0 0.0 0.0 0.0 0.0 ... nan nan nan nan
    palette   (time, rgb, eightbitcolor) float64 91.0 10.0 118.0 ... 0.0 0.0 0.0
Attributes: (12/61)
    product_name:                     AQUA_MODIS.20030101.L3m.DAY.SST.x_sst.nc
    instrument:                       MODIS
    title:                            MODISA Level-3 Equidistant Cylindrical ...
    project:                          Ocean Biology Processing Group (NASA/GS...
    platform:                         Aqua
    source:                           satellite observations from MODIS-Aqua
    ...                               ...
    processing_level:                 L3 Mapped
    cdm_data_type:                    grid
    proj4_string:                     +proj=eqc +lat_ts=0 +lat_0=0 +x_0=0 +y_...
    data_bins:                        11567
    data_minimum:                     21.74
    data_maximum:                     28.5775

In [5]:
#temp = xr.open_dataset(path + "MODIS_temp_extended.nc").convert_calendar("noleap")

temp = temp.sel(time=~((temp.time.dt.month == 12) & (temp.time.dt.day >= 27))).resample(time = "1D").asfreq()

t = temp.where(temp["time.year"] == 2003, drop = True).resample(time = "8D").mean()
for y in range(2004,2022):
    t = xr.merge([t, temp.where(temp["time.year"] == y, drop = True).resample(time = "8D").mean()])

t = t.sst

datetimeindex = t.indexes['time'].to_datetimeindex()

t["time"] = datetimeindex

c:\Users\feder\miniconda3\lib\site-packages\xarray\coding\times.py:351: FutureWarning: Index.ravel returning ndarray is deprecated; in a future version this will return a view on self.
  sample = dates.ravel()[0]
C:\Users\feder\AppData\Local\Temp\ipykernel_6004\83018299.py:11: RuntimeWarning: Converting a CFTimeIndex with dates from a non-standard calendar, 'noleap', to a pandas.DatetimeIndex, which uses dates from the standard calendar.  This may lead to subtle errors in operations that depend on the length of time between dates.
  datetimeindex = t.indexes['time'].to_datetimeindex()


In [6]:
t

<xarray.DataArray 'sst' (time: 873, lat: 240, lon: 144)>
array([[[23.789999, 23.699999, 23.605   , ..., 24.404999, 24.48    ,
         24.369999],
        [23.664999, 23.664999, 23.785   , ..., 24.494999, 24.3     ,
         24.185   ],
        [23.769999, 23.795   , 23.822498, ..., 24.345   , 24.32    ,
         24.475   ],
        ...,
        [25.57    , 25.75    , 25.76    , ...,       nan,       nan,
               nan],
        [25.595   , 25.564999, 25.625   , ..., 28.172499,       nan,
         27.859999],
        [25.625   , 25.689999, 25.279999, ..., 28.2825  , 28.515   ,
         28.67    ]],

       [[23.789999, 23.409998, 23.288332, ..., 23.704998, 23.414999,
         23.5     ],
        [23.718332, 23.486666, 23.338333, ..., 23.704998, 23.57    ,
         23.635   ],
        [23.771666, 23.611666, 23.46    , ..., 23.739998, 23.585   ,
         23.58    ],
...
        [      nan,       nan,       nan, ...,       nan,       nan,
               nan],
        [      nan,       nan,       nan, ...,       nan,       nan,
               nan],
        [      nan,       nan,       nan, ...,       nan,       nan,
               nan]],

       [[      nan,       nan,       nan, ...,       nan,       nan,
         24.785   ],
        [      nan,       nan,       nan, ...,       nan,       nan,
         24.76    ],
        [      nan,       nan,       nan, ...,       nan,       nan,
         24.84    ],
        ...,
        [25.965   , 25.96    , 25.731667, ..., 28.773333, 28.662498,
         28.849998],
        [25.9275  , 24.928335, 24.99125 , ..., 28.69    , 28.663332,
         28.724998],
        [25.925001, 26.09    , 25.973333, ..., 28.494999, 28.57    ,
         28.727499]]], dtype=float32)
Coordinates:
  * time     (time) datetime64[ns] 2003-01-01 2003-01-09 ... 2021-12-19
  * lat      (lat) float32 18.98 18.94 18.9 18.85 ... 9.146 9.104 9.062 9.021
  * lon      (lon) float32 108.0 108.1 108.1 108.1 ... 113.9 113.9 113.9 114.0

In [7]:
t.load().to_netcdf(path + "MODIS_temp_extended_8D.nc")